Going to play with different Neural Networks to try to model all of the data. I don't seem to have saved a varient of the dataframe where all of the stations are present

In [6]:
import polars as pl
from tqdm import tqdm
import os
import pickle

In [21]:
# use CSVs in dir
# file_path = f"data/tfl_CSVs/"
directory = f"data/Journey_Data/"
downloaded_files = set(os.listdir(directory))
CSVs = downloaded_files
print(f"There are {len(CSVs)} files: {CSVs}")

There are 23 files: {'06JourneyDataExtract18May2016-24May2016.csv', '100JourneyDataExtract07Mar2018-13Mar2018.csv', '104JourneyDataExtract04Apr2018-10Apr2018.csv', '109JourneyDataExtract09May2018-15May2018.csv', '107JourneyDataExtract25Apr2018-01May2018.csv', '05JourneyDataExtract01May2016-17May2016.csv', '03JourneyDataExtract06Mar2016-31Mar2016.csv', '105JourneyDataExtract11Apr2018-17Apr2018.csv', '07JourneyDataExtract25May2016-31May2016.csv', '10a-Journey-Data-Extract-20Sep15-03Oct15.csv', '10JourneyDataExtract15Jun2016-21Jun2016.csv', '103JourneyDataExtract28Mar2018-03Apr2018.csv', '04JourneyDataExtract01Apr2016-30Apr2016.csv', '01aJourneyDataExtract10Jan16-23Jan16.csv', '02bJourneyDataExtract21Feb16-05Mar2016.csv', '02aJourneyDataExtract07Fe16-20Feb2016.csv', '102JourneyDataExtract21Mar2018-27Mar2018.csv', '101JourneyDataExtract14Mar2018-20Mar2018.csv', '01bJourneyDataExtract24Jan16-06Feb16.csv', '108JourneyDataExtract02May2018-08May2018.csv', '08JourneyDataExtract01Jun2016-07Jun20

In [22]:
# Initialize an empty Polars DataFrame
merged_df = pl.DataFrame()
cols = []

progress_bar = tqdm(CSVs, desc="Processing")

for idx, csv in enumerate(progress_bar):
    file_path = f"{directory}{csv}"
    print(f"CSV: {csv}\n file_path: {file_path}")
    df = pl.read_csv(file_path, skip_rows_after_header=1, ignore_errors=True)
    # print(df.columns)

    # Rename columns and keep only 'Rental ID', 'Start Date', 'Start Station Name', 'End Date', 'End Station Name'
    renaming_dict = {
        'Number': 'Rental ID',
        'rental ID': 'Rental ID',
        'Rental Id': 'Rental ID',
        'Start date': 'Start Date',
        'StartStation Name': 'Start Station Name',
        'Start station': 'Start Station Name',
        'End date': 'End Date',
        'EndStation Name': 'End Station Name',
        'End station': 'End Station Name'
    }

    for old_col, new_col in renaming_dict.items():
        if old_col in df.columns:
            df = df.rename({old_col: new_col})
    
    cols_to_keep = ['Rental ID', 'Start Date', 'Start Station Name', 'End Date', 'End Station Name']
    df = df.select(cols_to_keep)

    # Convert date columns
    df = df.with_columns([
        pl.col('Start Date').str.strptime(pl.Datetime, strict=False),
        pl.col('End Date').str.strptime(pl.Datetime, strict=False),
        pl.col('Start Station Name').cast(pl.Utf8),
        pl.col('End Station Name').cast(pl.Utf8)
    ])

    # Filter rows (if needed)
    # df = df.filter((pl.col("Start Station Name") == "Waterloo Station 3, Waterloo") |
    #                (pl.col("End Station Name") == "Waterloo Station 3, Waterloo"))
    
    # Concatenate DataFrames
    merged_df = pl.concat([merged_df, df])

    progress_bar.set_description(
        f"Processing Index: {idx+1}/{len(CSVs)}, merged_df is {merged_df.estimated_size()/(1024*1024)}MB large"
    )

Processing Index: 1/23, merged_df is 16.760796546936035MB large:   4%|▍         | 1/23 [00:00<00:02,  9.09it/s]

CSV: 06JourneyDataExtract18May2016-24May2016.csv
 file_path: data/Journey_Data/06JourneyDataExtract18May2016-24May2016.csv
CSV: 100JourneyDataExtract07Mar2018-13Mar2018.csv
 file_path: data/Journey_Data/100JourneyDataExtract07Mar2018-13Mar2018.csv


Processing Index: 4/23, merged_df is 57.76355171203613MB large:  13%|█▎        | 3/23 [00:00<00:01, 11.31it/s] 

CSV: 104JourneyDataExtract04Apr2018-10Apr2018.csv
 file_path: data/Journey_Data/104JourneyDataExtract04Apr2018-10Apr2018.csv
CSV: 109JourneyDataExtract09May2018-15May2018.csv
 file_path: data/Journey_Data/109JourneyDataExtract09May2018-15May2018.csv
CSV: 107JourneyDataExtract25Apr2018-01May2018.csv
 file_path: data/Journey_Data/107JourneyDataExtract25Apr2018-01May2018.csv


Processing Index: 5/23, merged_df is 70.63428020477295MB large:  22%|██▏       | 5/23 [00:00<00:01, 11.54it/s]

...

Processing Index: 19/23, merged_df is 408.36723041534424MB large:  78%|███████▊  | 18/23 [00:02<00:00, 10.37it/s]

CSV: 01bJourneyDataExtract24Jan16-06Feb16.csv
 file_path: data/Journey_Data/01bJourneyDataExtract24Jan16-06Feb16.csv
CSV: 108JourneyDataExtract02May2018-08May2018.csv
 file_path: data/Journey_Data/108JourneyDataExtract02May2018-08May2018.csv


Processing Index: 22/23, merged_df is 461.58660888671875MB large:  96%|█████████▌| 22/23 [00:02<00:00,  9.89it/s]

CSV: 08JourneyDataExtract01Jun2016-07Jun2016.csv
 file_path: data/Journey_Data/08JourneyDataExtract01Jun2016-07Jun2016.csv
CSV: 09JourneyDataExtract08Jun2016-14Jun2016.csv
 file_path: data/Journey_Data/09JourneyDataExtract08Jun2016-14Jun2016.csv
CSV: 106JourneyDataExtract18Apr2018-24Apr2018.csv
 file_path: data/Journey_Data/106JourneyDataExtract18Apr2018-24Apr2018.csv


Processing Index: 23/23, merged_df is 482.20048904418945MB large: 100%|██████████| 23/23 [00:02<00:00,  9.01it/s]


In [33]:
pickle_dir = "pickle/df_temp"

# # save to pickle
# with open(pickle_dir, 'wb') as f:
#     pickle.dump(df, f)

#load from pickle
with open(pickle_dir, 'rb') as f:
    df = pickle.load(f)

df.head()

Rental ID,Start Date,Start Station Name,End Date,End Station Name
i64,datetime[μs],str,datetime[μs],str
74656682,2018-04-18 07:32:00,"""Crosswall, Tower""",2018-04-18 07:43:00,"""Tallis Street, Temple"""
74699122,2018-04-19 07:31:00,"""Crosswall, Tower""",2018-04-19 07:42:00,"""Tallis Street, Temple"""
74831922,2018-04-21 22:42:00,"""Moor Street, Soho""",2018-04-21 23:14:00,"""Geraldine Street, Elephant & C…"
74740907,2018-04-19 22:51:00,"""Moor Street, Soho""",2018-04-19 23:10:00,"""Walworth Road, Elephant & Cast…"
74787286,2018-04-20 22:31:00,"""Moor Street, Soho""",2018-04-20 22:46:00,"""Ontario Street, Elephant & Cas…"


In [57]:
start_stations = df.select(pl.col('Start Station Name').alias('Station Name'))
end_stations = df.select(pl.col('End Station Name').alias('Station Name'))

unique_values = (
    pl.concat([start_stations, end_stations])
    .unique()
    .sort("Station Name")
)
id_mapping = unique_values.with_row_count("ID")
print(id_mapping)

shape: (786, 2)
┌─────┬─────────────────────────────────┐
│ ID  ┆ Station Name                    │
│ --- ┆ ---                             │
│ u32 ┆ str                             │
╞═════╪═════════════════════════════════╡
│ 0   ┆ Abbey Orchard Street, Westmins… │
│ 1   ┆ Abbotsbury Road, Holland Park   │
│ 2   ┆ Aberdeen Place, St. John's Woo… │
│ 3   ┆ Aberfeldy Street, Poplar        │
│ 4   ┆ Abingdon Green, Westminster     │
│ …   ┆ …                               │
│ 781 ┆ Wormwood Street, Liverpool Str… │
│ 782 ┆ Wren Street, Holborn            │
│ 783 ┆ Wright's Lane, Kensington       │
│ 784 ┆ Wynne Road, Stockwell           │
│ 785 ┆ York Hall, Bethnal Green        │
└─────┴─────────────────────────────────┘


/tmp/ipykernel_41762/363409236.py:9: DeprecationWarning: `DataFrame.with_row_count` is deprecated. Use `with_row_index` instead. Note that the default column name has changed from 'row_nr' to 'index'.
  id_mapping = unique_values.with_row_count("ID")


In [60]:
df_with_ids = df.join(id_mapping.rename({"Station Name": "Start Station Name", "ID": "Start_ID"}), on="Start Station Name", how="left")
df_with_ids = df_with_ids.join(id_mapping.rename({"Station Name": "End Station Name", "ID": "End_ID"}), on="End Station Name", how="left")

df_with_ids.head()

Rental ID,Start Date,Start Station Name,End Date,End Station Name,Start_ID,End_ID
i64,datetime[μs],str,datetime[μs],str,u32,u32
74656682,2018-04-18 07:32:00,"""Crosswall, Tower""",2018-04-18 07:43:00,"""Tallis Street, Temple""",182,684
74699122,2018-04-19 07:31:00,"""Crosswall, Tower""",2018-04-19 07:42:00,"""Tallis Street, Temple""",182,684
74831922,2018-04-21 22:42:00,"""Moor Street, Soho""",2018-04-21 23:14:00,"""Geraldine Street, Elephant & C…",452,268
74740907,2018-04-19 22:51:00,"""Moor Street, Soho""",2018-04-19 23:10:00,"""Walworth Road, Elephant & Cast…",452,731
74787286,2018-04-20 22:31:00,"""Moor Street, Soho""",2018-04-20 22:46:00,"""Ontario Street, Elephant & Cas…",452,496


In [62]:
df_brief = df_with_ids["Rental ID", "Start Date", "End Date", "Start_ID", "End_ID"]
df_brief.head()

Rental ID,Start Date,End Date,Start_ID,End_ID
i64,datetime[μs],datetime[μs],u32,u32
74656682,2018-04-18 07:32:00,2018-04-18 07:43:00,182,684
74699122,2018-04-19 07:31:00,2018-04-19 07:42:00,182,684
74831922,2018-04-21 22:42:00,2018-04-21 23:14:00,452,268
74740907,2018-04-19 22:51:00,2018-04-19 23:10:00,452,731
74787286,2018-04-20 22:31:00,2018-04-20 22:46:00,452,496


In [69]:
df_start = df_brief["Rental ID", "Start Date", "Start_ID"].rename({"Start Date": "Date", "Start_ID": "ID"}).with_columns(pl.lit(-1).alias("flow"))

df_end = df_brief["Rental ID", "End Date", "End_ID"].rename({"End Date": "Date", "End_ID": "ID"}).with_columns(pl.lit(1).alias("flow"))

df_flows = pl.concat([df_start, df_end]).sort("Date")
df_flows.head(50)

Rental ID,Date,ID,flow
i64,datetime[μs],u32,i32
74654037,2018-04-18 00:00:00,551,-1
74654041,2018-04-18 00:00:00,204,-1
74654038,2018-04-18 00:00:00,552,-1
74654036,2018-04-18 00:00:00,670,-1
74654039,2018-04-18 00:00:00,695,-1
…,…,…,…
74654057,2018-04-18 00:10:00,587,1
74654066,2018-04-18 00:10:00,361,1
74654049,2018-04-18 00:10:00,83,1
